In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route length
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

In [5]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route length
        result.append(s['CO2'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [6]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 25 datapoints from the parameter space:

In [7]:
init_X, init_Y = pickle.load(open('experimental_design/init_points/20_init_points.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 5
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
with open(f'experimental_design/init_points/{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)

'''

'\ndesign = RandomDesign(parameter_space)  # initialize with random points\nnum_init_points = 5\ninit_X = design.get_samples(num_init_points)\ninit_Y = user_function_time_loss(init_X)\nwith open(f\'experimental_design/init_points/{num_init_points}_init_points.pkl\', "wb") as f:\n      pickle.dump((init_X, init_Y), f)\n\n'

To then fit a GP to these points

In [8]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = -26.105669864682376
Optimization restart 2/5, f = -0.6890488010395366
Optimization restart 3/5, f = -26.584518930804258
Optimization restart 4/5, f = -26.584518930801593
Optimization restart 5/5, f = -0.6890488010479441


### Experimentally designed emulator

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_exp_design = pickle.load(open('experimental_design/models/20_init_200_loop_model_timeLoss.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 40

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_exp_design = ExpectedImprovement(emukit_model_exp_design)

### Optimisation loop

In [13]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -26.584518930804258
Optimization restart 2/5, f = -0.6890488010479476
Optimization restart 3/5, f = -26.584518930719845
Optimization restart 4/5, f = -0.6890488010479423
Optimization restart 5/5, f = -26.58451893073743
[[20.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -27.499763740132007
Optimization restart 2/5, f = -27.499763740168248
Optimization restart 3/5, f = -1.1639568927650537
Optimization restart 4/5, f = -27.499763739502818
Optimization restart 5/5, f = -1.1639569050840013
[[20.   8.  27.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


 Retrying in 1 seconds
Optimization restart 1/5, f = -27.8640419278819
Optimization restart 2/5, f = -1.6280616669000363
Optimization restart 3/5, f = -1.6280615058189625
Optimization restart 4/5, f = -1.6280663361467624
Optimization restart 5/5, f = -27.86404192788233
[[20.         25.          5.         54.28963798  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -25.04698199407387
Optimization restart 2/5, f = -1.5916955707605531
Optimization restart 3/5, f = -1.5916955707605904
Optimization restart 4/5, f = -1.5916955707605585
Optimization restart 5/5, f = -1.5916955707605567
[[20.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -26.763613846571886
Optimization restart 2/5, f = -9.53652363773706
Optimization restart 3/5, f = -9.536523855491907
Optimiza

 Retrying in 1 seconds
Optimization restart 1/5, f = -39.9633575419137
Optimization restart 2/5, f = -39.96335754226081
Optimization restart 3/5, f = -9.611478192876621
Optimization restart 4/5, f = -49.983663124645005
Optimization restart 5/5, f = -9.61147819252967
[[ 3.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -52.04154922214627
Optimization restart 2/5, f = -17.556306443832945
Optimization restart 3/5, f = -17.556309963641713
Optimization restart 4/5, f = -17.556306479915346
Optimization restart 5/5, f = -17.556306378608433
[[ 3.          8.          5.         61.61761458  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -54.163847534532664
Optimization restart 2/5, f = -18.01210328265024
Optimization restart 3/5, f = -50.52950219991506
Optimizati

In [14]:
bo_exp_design = BayesianOptimizationLoop(parameter_space, emukit_model_exp_design, acquisition=acquisition_exp_design)
bo_exp_design.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -334.39223493189
Optimization restart 2/5, f = -302.15417480448195
Optimization restart 3/5, f = -334.39223493016357
Optimization restart 4/5, f = -334.3922348316994
Optimization restart 5/5, f = -334.3922349318833
[[14.         15.38636954  5.         58.16069822  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -336.75880841020137
Optimization restart 2/5, f = -304.48986198227226
Optimization restart 3/5, f = -304.4894979317078
Optimization restart 4/5, f = -304.48986299337776
Optimization restart 5/5, f = -336.7588084064274
[[11.         12.27323359  5.         58.07083153  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -339.13836142237017
Optimization restart 2/5, f = 48.97936567624845
Optimization restart 3/5, f = -307.351485

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = -356.5918647925047
[[15.         8.         6.9217836 70.         1.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -359.2003898531053
Optimization restart 2/5, f = -359.20038985186267
Optimization restart 3/5, f = -359.2003898436295
Optimization restart 4/5, f = -359.2003898531485
Optimization restart 5/5, f = -359.2003898526981
[[15.         13.60444393  5.         70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -361.78728250032896
Optimization restart 2/5, f = -361.78728250027734
Optimization restart 3/5, f = -361.7872825015615


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -359.0327214197803
Optimization restart 5/5, f = -361.7872825008721
[[16.          8.          7.21797728 70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -364.4922760524562
Optimization restart 2/5, f = -364.4922760524935
Optimization restart 3/5, f = -364.4922760471213
Optimization restart 4/5, f = -364.49227605095496
Optimization restart 5/5, f = -364.4922760514603
[[16.         12.79369087  5.         70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -367.1889802810323
Optimization restart 2/5, f = -367.1889802844545
Optimization restart 3/5, f = -367.18898026734905
Optimization restart 4/5, f = -367.18898028440856
Optimization restart 5/5, f = -367.1889802837938
[[14.         25.          5.         54.

 Retrying in 1 seconds
Optimization restart 1/5, f = -415.7288558926008
Optimization restart 2/5, f = -404.55364744043607
Optimization restart 3/5, f = -415.7289726354183
Optimization restart 4/5, f = -404.55364744060205
Optimization restart 5/5, f = -415.7289726480908
[[17.         15.28694672  5.         54.8082942   1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -418.6405996443881
Optimization restart 2/5, f = -192.23169771957816
Optimization restart 3/5, f = -409.8474928808729
Optimization restart 4/5, f = -409.8474928798865
Optimization restart 5/5, f = -418.64059963996044
[[20.         11.19624528  5.         70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -421.527722300903
Optimization restart 2/5, f = -415.1363815838982
Optimization resta

## Results

### Results on the random initialised emulator

In [15]:
results_random_init = bo_random_init.get_results()

In [16]:
results_random_init.minimum_location

array([20.        ,  8.        ,  5.        , 61.72224432,  1.        ,
        1.5       ])

In [21]:
results_random_init.minimum_value*100

8.269701772078951

### Results on the experimentally designed emulator

In [18]:
results_exp_design = bo_exp_design.get_results()

In [19]:
results_exp_design.minimum_location

array([17.        ,  8.        ,  6.85566823, 70.        ,  1.        ,
        1.5       ])

In [22]:
results_exp_design.minimum_value*100

7.926160594614354